In [3]:
import sys
at_colab = "google.colab" in sys.modules

In [4]:
if at_colab:
    !pip install fastkml
    !pip install geopandas
    !pip install rasterio
    !pip install gadm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 861 kB/s 
  Created wheel for pygeoif: filename=pygeoif-0.7-py3-none-any.whl size=19249 sha256=fa103b92dc250938af8d3618a8f73dc68d91652b3901b0bc204ac993f3cd6a8d
  Stored in directory: /root/.cache/pip/wheels/4a/84/19/a1fcaf92f8f57a424eca18e2d8c03d149436806b91474bcb89
Successfully built pygeoif
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.0 MB/s 
     |████████████████████████████████| 6.3 MB 28.5 MB/s 
     |████████████████████████████████| 16.7 MB 345 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 81 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████

In [5]:
import requests, urllib, fastkml, geopandas as gp, folium, rasterio, rasterio.mask
from gadm import GADMDownloader
from branca.element import Figure

## Administrative data

In [6]:
downloader = GADMDownloader(version='4.0')
kenya = downloader.get_shape_data_by_country_name(country_name='Kenya', ad_level=2)

100%|██████████| 47.8M/47.8M [00:32<00:00, 1.45MB/s]


## The MrGreen data

Pank Seelen sent this:

---

Hi Samuel,

Hereby a [link to Google](https://eur04.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.google.com%2Fmaps%2Fd%2Fu%2F0%2Fedit%3Fmid%3D1uBPMv1UM4wOshDvQf48AYzGX4iOivX0%26usp%3Dsharing&data=05%7C01%7Cj.a.s.gromicho%40uva.nl%7C37a74c6538324f218f9608dac3d9e359%7Ca0f1cacd618c4403b94576fb3d6874e5%7C0%7C0%7C638037637853280029%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C3000%7C%7C%7C&sdata=T7P4IWfCQcln6xz14Qq6dXmp1fgTmn%2BkfLmAAz7LRCk%3D&reserved=0) maps with all our collection locations:
- Trading Points: the locations where we engage with traditional waste pickers to buy plastic from them
- Total model: Partnerships with TotalEnergies, where we currently collect plastic from consumers at two petrol stations (plans to increase this number to 10 early next year)
- Duka model: Duka shops that we facilitate to collect plastic from consumers

Let me know if you have any questions.

Pank

---

Below we download that data in [KML](https://en.wikipedia.org/wiki/Keyhole_Markup_Language).

In [7]:
MID='1uBPMv1UM4wOshDvQf48AYzGX4iOivX0'
kml_url=f'https://www.google.com/maps/d/u/0/kml?mid={MID}&forcekml=1'

In [8]:
kml = fastkml.kml.KML()
kml.from_string( requests.get(kml_url).content )

Now we collect the coordinates into a dictionary indexed on the name. 

In [9]:
def Collect( store, element ):
    if getattr(element, 'features', None):
        for feature in element.features():
            Collect( store, feature)
    elif getattr(element, 'geometry', None):
        store[element.name] = (element.geometry.x,element.geometry.y)
    else: 
        pass

In [10]:
kml_data = dict()
Collect(kml_data, kml)

And we put it into a dataframe.

In [11]:
mrgreen = gp.GeoDataFrame.from_dict( kml_data, orient='index' )
mrgreen.columns = ['lon','lat']

In [12]:
mrgreen

,lon,lat
Factory,36.865515,-1.328379
02 - Pipeline,36.890782,-1.324501
08 - Eastleigh B,36.848944,-1.266957
12 - Kawangware,36.748210,-1.287301
16 - Kayole A,36.922235,-1.269406
18 - Kayole B,36.907241,-1.287062
19 - Kibera Ayany,36.773055,-1.309143
20 - Kibera Lindi,36.792899,-1.315145
29 - Dandora,36.888039,-1.257197
New TP (Tassia),36.898298,-1.306064


## Population data

In [13]:
%%time
country = 'KEN'
worldpop_url = f'https://www.worldpop.org/rest/data/pop/wpgpunadj/?iso3={country}'
headers = {'User-Agent': 'Chrome/107.0.5304.107'}  # User-Agent is required to get API response
response = requests.get(worldpop_url, headers=headers)

# Extract url for data for the latest year
data = response.json()['data'][-1]
url = data['files'][0]
filehandle, _ = urllib.request.urlretrieve(url)

CPU times: user 6.88 s, sys: 3.14 s, total: 10 s
Wall time: 9min 36s


In [14]:
url

'https://data.worldpop.org/GIS/Population/Global_2000_2020/2020/KEN/ken_ppp_2020_UNadj.tif'

We got WorldPop data for year 2020.

In [15]:
src = rasterio.open(filehandle)
raster_tot = src.read(1)
raster_tot[raster_tot<0] = None

In [16]:
raster_nonzero = raster_tot[raster_tot>0]
population_worldpop = raster_nonzero.sum()
print(round(population_worldpop/1000000,2),'million')

53.77 million


In [17]:
def get_population_count(vector_polygon,raster_layer):
    gtraster, _ = rasterio.mask.mask(raster_layer, [vector_polygon], crop=True)
    return gtraster[0][gtraster[0]>0].sum()

In [18]:
kenya['population'] = [ get_population_count(g,src) for g in kenya.geometry ]
nairobi = kenya[kenya.NAME_1=='Nairobi']

In [19]:
nairobi.population.sum()

4695099.0

## Showing the data so far

In [20]:
fig = Figure(width=800, height=600)

a,b,c,d = nairobi.geometry.unary_union.bounds 
start_coords = ((d-b)/2,(c-a)/2)
fm = folium.Map(location=start_coords)
fm.fit_bounds( ((b,a), (d,c)) )
folium.Choropleth(
    geo_data=nairobi,
    name="population count",
    data=nairobi,
    columns= ["NAME_2","population"],
    key_on="feature.properties.NAME_2",
    fill_color="Reds",
    fill_opacity=0.65,
    line_opacity=0.2,
    legend_name="Total Population in area NAME_2").add_to(fm)
fg=folium.FeatureGroup(name='MrGreen Points', show=True)
fm.add_child(fg)
for name,x,y in mrgreen.itertuples(index=True):
    folium.Marker(location=(y,x),popup=name).add_to(fg)
folium.LayerControl().add_to(fm)    
fig.add_child(fm)

In [ ]:
#Proximity Analysis

In [22]:
print(mrgreen.crs)

None


In [ ]:
MID='1uBPMv1UM4wOshDvQf48AYzGX4iOivX0'
kml_url_2=f'https://goo.gl/maps/Fe5kvofpmddQT2X98'

In [ ]:
kml2 = fastkml.kml.KML()
kml2.from_string( requests.get(kml_url_2).content )

In [ ]:
kml_data2 = dict()
Collect(kml_data2, kml2)

In [ ]:
dumpsites = gp.GeoDataFrame.from_dict( kml_data2, orient='index' )
dumpsites.columns = ['lon','lat']

In [ ]:
# Select one collection incident in particular
collection_point = mrgreen.iloc[360]

# Measure distance from site to each station
distances = mrgreen.geometry.distance(collection_point.geometry)
distances

In [ ]:
print('Mean distance to collection stations: {} feet'.format(distances.mean()))

In [ ]:
print('Closest collection station ({} feet):'.format(distances.min()))
print(stations.iloc[distances.idxmin()][["ADDRESS", "LATITUDE", "LONGITUDE"]])